# WEO Economic Data Analysis & Recession Prediction

**Objective:** Load World Economic Outlook (WEO) data, clean and transform it, then use machine learning models to predict global recessions.

**Workflow:**
1. Data loading and cleaning
2. Feature engineering and recession flagging
3. Exploratory data analysis
4. Model training with full and reduced feature sets (comparing 13 vs 5 features)
5. Economy-specific analysis (Upper vs Lower economies with both feature sets)
6. Future predictions for all scenarios

**Models Used:** Logistic Regression, Random Forest, Gradient Boosting, Linear SVM, Decision Tree, and Ensemble

In [ ]:
# Core data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# File handling
import csv
from pathlib import Path

# Machine learning - preprocessing
from sklearn.preprocessing import StandardScaler

# Machine learning - models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Machine learning - metrics
from sklearn.metrics import (
    classification_report, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix,
    ConfusionMatrixDisplay,
    roc_curve,
    auc
)

# Imbalanced learning
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Optional pycountry for continent mapping
try:
    import pycountry
    import pycountry_convert as pc
    HAS_PYCOUNTRY = True
except ImportError:
    HAS_PYCOUNTRY = False

pd.set_option("display.max_rows", 25)

# 1. Data Loading

In [ ]:
p = Path(r"data.csv")
if not p.exists():
    raise FileNotFoundError(p)

# Detect encoding and delimiter
encoding = "utf-8"
try:
    sample = p.read_text(encoding=encoding)[:8192]
except UnicodeDecodeError:
    encoding = "latin-1"
    sample = p.read_text(encoding=encoding)[:8192]

try:
    delim = csv.Sniffer().sniff(sample).delimiter
except Exception:
    delim = ","

df = pd.read_csv(p, sep=delim, encoding=encoding, low_memory=False, parse_dates=True)
print("Shape:", df.shape)
df.head()

In [ ]:
print(f"Number of columns: {df.shape[1]}")
print(f"Number of rows: {df.shape[0]}")
print("\nColumn names:", df.columns.tolist())

In [ ]:
df.drop(columns=["WEO Country Code", "ISO", "Country/Series-specific Notes", "Subject Notes", 
                 "Units", "Scale", "Estimates Start After", "Subject Descriptor"], inplace=True)

codes = {
    # Core growth & external
    "NGDP_RPCH", "NGDPRPC", "PCPIPCH", "TX_RPCH", "TM_RPCH", "BCA_NGDPD",
    # Fiscal & debt aggregates
    "GGR_NGDP", "GGX_NGDP", "GGXWDN_NGDP", "GGXWDG_NGDP",
    # Savings & investment
    "NGSD_NGDP", "NID_NGDP",
    # Prices
    "PCPI", "LUR"
}

col = "WEO Subject Code"

if col not in df.columns:
    raise KeyError(f"Column {col!r} not found in dataframe")

df = df[df[col].astype(str).str.strip().isin(codes)].copy()
print("shape after filter:", df.shape)
df

## Data Reshaping: Wide to Long to Wide

In [ ]:
year_cols = df.columns[2:]

df[year_cols] = df[year_cols].replace({',': ''}, regex=True)
df[year_cols] = df[year_cols].apply(pd.to_numeric, errors="coerce")

df["Country"] = (
    df["Country"]
    .str.replace(" ", "_")
    .str.replace("'", "")
    .str.replace("-", "_")
)

df_long = df.melt(id_vars=["WEO Subject Code", "Country"],
                  var_name="Year", value_name="Value")

df_long["Year"] = df_long["Year"].astype(str).str.strip()
df_long = df_long[df_long["Year"].str.fullmatch(r"\d{4}")].copy()
df_long["Year"] = df_long["Year"].astype(int)

df_long["Value"] = (
    df_long["Value"].astype(str)
    .str.replace(",", "")
    .replace({"": None, "nan": None})
    .astype(float)
)

df_pivot = df_long.pivot_table(
    index=["Country", "Year"],
    columns="WEO Subject Code",
    values="Value",
    aggfunc="first"
).reset_index()

df_pivot.columns.name = None
df_pivot = df_pivot.set_index("Year")

df_pivot

# 3. Feature Engineering

## Add Recession Target Variable

In [ ]:
# --- Step 1: Ensure chronological order ---
df_pivot = df_pivot.sort_index()

# --- Step 2: Construct diagnostic flags (not stored in df) ---

# 1. GDP-based recession (two consecutive annual declines)
flag_gdp = (
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change() < 0)) &
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change().shift(-1) < 0))
).astype(int)

# 2. Investment collapse (sharp drop in investment)
flag_invest = (
    df_pivot.groupby("Country")["NID_NGDP"].transform(lambda x: x.diff() < -2)
).astype(int)

# 3. Savings decline (household/national savings falling)
flag_savings = (
    df_pivot.groupby("Country")["NGSD_NGDP"].transform(lambda x: x.diff() < -2)
).astype(int)

# 4. Trade shock (both exports and imports contracting)
flag_trade = (
    (df_pivot.groupby("Country")["TX_RPCH"].transform(lambda x: x < 0)) &
    (df_pivot.groupby("Country")["TM_RPCH"].transform(lambda x: x < 0))
).astype(int)

# 5. Inflation shock (stagflation: high inflation + negative growth)
flag_inflation = (
    (df_pivot.groupby("Country")["PCPIPCH"].transform(lambda x: x > 10)) &
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change() < 0))
).astype(int)

# 6. Debt crisis (gross debt rising sharply above 90% of GDP)
flag_debt = (
    (df_pivot["GGXWDG_NGDP"] > 90) &
    (df_pivot.groupby("Country")["GGXWDG_NGDP"].transform(lambda x: x.diff() > 10))
).astype(int)

# 7. Fiscal crisis (large and growing deficit)
flag_fiscal = (
    (df_pivot.groupby("Country")["GGR_NGDP"].transform(lambda x: x) < 
     df_pivot.groupby("Country")["GGX_NGDP"].transform(lambda x: x) - 5) &  # Deficit > 5% GDP
    (df_pivot.groupby("Country")["GGX_NGDP"].transform(lambda x: x.diff()) > 2)  # Rising spending
).astype(int)

# 8. Current account crisis (large deficit deteriorating)
flag_current_account = (
    (df_pivot["BCA_NGDPD"] < -5) &  # Deficit > 5% of GDP
    (df_pivot.groupby("Country")["BCA_NGDPD"].transform(lambda x: x.diff() < -2))  # Worsening
).astype(int)

# 9. Real GDP growth collapse (severe contraction)
flag_growth_collapse = (
    df_pivot.groupby("Country")["NGDP_RPCH"].transform(lambda x: x < -3)  # Growth < -3%
).astype(int)

# 10. Deflation risk (falling prices with economic weakness)
flag_deflation = (
    (df_pivot.groupby("Country")["PCPIPCH"].transform(lambda x: x < 0)) &  # Negative inflation
    (df_pivot.groupby("Country")["NGDP_RPCH"].transform(lambda x: x < 1))  # Weak growth
).astype(int)

# 11. Credit crunch (investment falling while debt rising)
flag_credit_crunch = (
    (df_pivot.groupby("Country")["NID_NGDP"].transform(lambda x: x.diff() < -1)) &
    (df_pivot.groupby("Country")["GGXWDG_NGDP"].transform(lambda x: x.diff() > 5))
).astype(int)

# 12. External shock (exports collapsing while imports stable/rising)
flag_external_shock = (
    (df_pivot.groupby("Country")["TX_RPCH"].transform(lambda x: x < -5)) &
    (df_pivot.groupby("Country")["TM_RPCH"].transform(lambda x: x > -2))
).astype(int)

# --- Step 3: Combine signals into a single severity score ---
local_signal_count = (
    flag_gdp + 
    flag_invest + 
    flag_savings + 
    flag_trade + 
    flag_inflation +
    flag_debt +
    flag_fiscal +
    flag_current_account +
    flag_growth_collapse +
    flag_deflation +
    flag_credit_crunch +
    flag_external_shock
)

# --- Step 4: Multiclass recession risk label ---
def classify_risk(local_count):
    """
    Same thresholds as original:
    - 3+ signals: High risk
    - 2 signals: Moderate risk  
    - 1 signal: Mild risk
    - 0 signals: No risk
    """
    if local_count >= 3:
        return 3  # High risk
    if local_count == 2:
        return 2  # Moderate risk
    if local_count == 1:
        return 1  # Mild risk
    return 0      # No risk

df_pivot["RecessionRisk"] = [
    classify_risk(l) for l in local_signal_count
]

# Store signal count
df_pivot["SignalCount"] = local_signal_count

# --- Step 5: Fill NaN with mean instead of dropping ---
# Fill numeric columns with their mean
numeric_cols = df_pivot.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    df_pivot[col] = df_pivot[col].fillna(df_pivot[col].mean())

# Fill non-numeric columns with forward fill or mode
non_numeric_cols = df_pivot.select_dtypes(exclude=[np.number]).columns
for col in non_numeric_cols:
    if col == 'Country':
        continue  # Don't fill Country column
    df_pivot[col] = df_pivot[col].fillna(method='ffill').fillna(method='bfill')

df_pivot = df_pivot.sort_index(ascending=True)

# --- Step 6: Show distribution ---
print("=" * 70)
print("RECESSION RISK CLASSIFICATION SUMMARY (12 Indicators)")
print("=" * 70)

print("\nRecessionRisk class distribution:")
print(df_pivot["RecessionRisk"].value_counts().sort_index())
print(f"\nTotal samples: {len(df_pivot)}")

# Show average signals per risk class
print("\n" + "=" * 70)
print("AVERAGE SIGNALS TRIGGERED PER RISK CLASS:")
print("=" * 70)

signal_summary = df_pivot.groupby('RecessionRisk')['SignalCount'].agg(['mean', 'min', 'max', 'count'])
print(signal_summary)

# Show which signals are most common
print("\n" + "=" * 70)
print("INDIVIDUAL SIGNAL FREQUENCIES:")
print("=" * 70)

# Recalculate flags for cleaned data
flag_gdp_clean = (
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change() < 0)) &
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change().shift(-1) < 0))
).astype(int)

flag_invest_clean = (
    df_pivot.groupby("Country")["NID_NGDP"].transform(lambda x: x.diff() < -2)
).astype(int)

flag_savings_clean = (
    df_pivot.groupby("Country")["NGSD_NGDP"].transform(lambda x: x.diff() < -2)
).astype(int)

flag_trade_clean = (
    (df_pivot.groupby("Country")["TX_RPCH"].transform(lambda x: x < 0)) &
    (df_pivot.groupby("Country")["TM_RPCH"].transform(lambda x: x < 0))
).astype(int)

flag_inflation_clean = (
    (df_pivot.groupby("Country")["PCPIPCH"].transform(lambda x: x > 10)) &
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change() < 0))
).astype(int)

flag_debt_clean = (
    (df_pivot["GGXWDG_NGDP"] > 90) &
    (df_pivot.groupby("Country")["GGXWDG_NGDP"].transform(lambda x: x.diff() > 10))
).astype(int)

flag_fiscal_clean = (
    (df_pivot.groupby("Country")["GGR_NGDP"].transform(lambda x: x) < 
     df_pivot.groupby("Country")["GGX_NGDP"].transform(lambda x: x) - 5) &
    (df_pivot.groupby("Country")["GGX_NGDP"].transform(lambda x: x.diff()) > 2)
).astype(int)

flag_current_account_clean = (
    (df_pivot["BCA_NGDPD"] < -5) &
    (df_pivot.groupby("Country")["BCA_NGDPD"].transform(lambda x: x.diff() < -2))
).astype(int)

flag_growth_collapse_clean = (
    df_pivot.groupby("Country")["NGDP_RPCH"].transform(lambda x: x < -3)
).astype(int)

flag_deflation_clean = (
    (df_pivot.groupby("Country")["PCPIPCH"].transform(lambda x: x < 0)) &
    (df_pivot.groupby("Country")["NGDP_RPCH"].transform(lambda x: x < 1))
).astype(int)

flag_credit_crunch_clean = (
    (df_pivot.groupby("Country")["NID_NGDP"].transform(lambda x: x.diff() < -1)) &
    (df_pivot.groupby("Country")["GGXWDG_NGDP"].transform(lambda x: x.diff() > 5))
).astype(int)

flag_external_shock_clean = (
    (df_pivot.groupby("Country")["TX_RPCH"].transform(lambda x: x < -5)) &
    (df_pivot.groupby("Country")["TM_RPCH"].transform(lambda x: x > -2))
).astype(int)

signal_names = [
    'GDP Decline', 'Investment Collapse', 'Savings Decline', 'Trade Shock',
    'Inflation Shock', 'Debt Crisis', 'Fiscal Crisis', 'Current Account Crisis',
    'Growth Collapse', 'Deflation Risk', 'Credit Crunch', 'External Shock'
]

signals_clean = [
    flag_gdp_clean, flag_invest_clean, flag_savings_clean, flag_trade_clean, 
    flag_inflation_clean, flag_debt_clean, flag_fiscal_clean, flag_current_account_clean,
    flag_growth_collapse_clean, flag_deflation_clean, flag_credit_crunch_clean, 
    flag_external_shock_clean
]

for name, signal in zip(signal_names, signals_clean):
    count = signal.sum()
    pct = (count / len(df_pivot)) * 100
    print(f"{name:<25} {count:>5} ({pct:>5.2f}%)")

print("\n" + "=" * 70)

# Remove temporary column
df_pivot = df_pivot.drop(columns=['SignalCount'])

# --- Output: dataframe with multiclass label ---
df_pivot

In [ ]:
# RecessionRisk Class Distribution - Compact Plot (up to 2025)
df_until_2025 = df_pivot[df_pivot.index <= 2025]
risk_counts = df_until_2025["RecessionRisk"].value_counts().sort_index()

plt.figure(figsize=(10, 6))
colors = ['green', 'yellow', 'orange', 'red']
plt.bar(risk_counts.index, risk_counts.values, color=colors, edgecolor='black', linewidth=1.5)
plt.xlabel('RecessionRisk Class', fontsize=12, fontweight='bold')
plt.ylabel('Count', fontsize=12, fontweight='bold')
plt.title('RecessionRisk Class Distribution (up to 2025)', fontsize=14, fontweight='bold')
plt.xticks([0, 1, 2, 3], ['No Risk', 'Mild', 'Moderate', 'High'])
plt.grid(axis='y', alpha=0.3)

# Add count labels on bars
for cls, count in risk_counts.items():
    plt.text(cls, count + 10, str(count), ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Total samples (up to 2025): {len(df_until_2025)}")

In [ ]:
# All RecessionRisk Classes by Year - Line Graph (up to 2025)
# Calculate counts for each class by year
df0 = df_until_2025[df_until_2025["RecessionRisk"] == 0].copy()
df1 = df_until_2025[df_until_2025["RecessionRisk"] == 1].copy()
df2 = df_until_2025[df_until_2025["RecessionRisk"] == 2].copy()
df3 = df_until_2025[df_until_2025["RecessionRisk"] == 3].copy()

class0_by_year = df0.groupby(df0.index).size()
class1_by_year = df1.groupby(df1.index).size()
class2_by_year = df2.groupby(df2.index).size()
class3_by_year = df3.groupby(df3.index).size()

# Get all years (up to 2025)
all_years = sorted(df_until_2025.index.unique())

# Reindex to include all years (fill missing with 0)
class0_by_year = class0_by_year.reindex(all_years, fill_value=0)
class1_by_year = class1_by_year.reindex(all_years, fill_value=0)
class2_by_year = class2_by_year.reindex(all_years, fill_value=0)
class3_by_year = class3_by_year.reindex(all_years, fill_value=0)

plt.figure(figsize=(14, 7))
plt.plot(all_years, class0_by_year.values, color='green', linewidth=2.5, marker='o', label='Class 0 (No Risk)', markersize=5)
plt.plot(all_years, class1_by_year.values, color='gold', linewidth=2.5, marker='s', label='Class 1 (Mild)', markersize=5)
plt.plot(all_years, class2_by_year.values, color='orange', linewidth=2.5, marker='^', label='Class 2 (Moderate)', markersize=5)
plt.plot(all_years, class3_by_year.values, color='red', linewidth=2.5, marker='D', label='Class 3 (High)', markersize=5)

plt.xlabel('Year', fontsize=12, fontweight='bold')
plt.ylabel('Number of Countries', fontsize=12, fontweight='bold')
plt.title('RecessionRisk Classes Over Time (up to 2030)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Cutting Out Data that leads to biasness & Review Remaining Countries

In [ ]:
df_filtered = df_pivot.loc[(df_pivot.index >= 2000) & (df_pivot.index <= 2025)]
df_filtered

In [ ]:
print(df_filtered["Country"].unique())


In [ ]:
df_filtered["Country"].nunique()


# 4. Exploratory Data Analysis

## Correlation Heatmap

## Prepare Features and Target

In [ ]:
X = df_filtered.drop(columns=["RecessionRisk", "Country"])
y = df_filtered["RecessionRisk"]

# 5. Machine Learning Models

## Global Dataset - Full Features (13 Features)

### Define and Train All Models

In [ ]:
# ============================================================
#             TRAIN ALL MODELS (MULTICLASS) - IMPROVED
# ============================================================
def train_all_models(X_train, y_train, X_val, y_val, X_test, y_test, model_params=None, use_xgb=False):
    """
    Enhanced model training with validation support.
    """
    if model_params is None:
        model_params = {
            'logit': {
                'C': 0.3,  # Increased regularization (lower C)
                'penalty': 'l2',
                'solver': 'saga',
                'max_iter': 10000,
                'random_state': 42,
                'class_weight': 'balanced'
            },
            'rf': {
                'n_estimators': 200,  # Reduced from 300
                'max_depth': 5,  # Reduced from 6
                'min_samples_leaf': 20,  # Increased from 15
                'min_samples_split': 40,  # Increased from 30
                'max_features': 'sqrt',
                'random_state': 42,
                'class_weight': 'balanced',
                'n_jobs': -1,
                'max_samples': 0.8  # Bootstrap with 80% samples
            },
            'gb': {
                'n_estimators': 150,  # Reduced from 300
                'learning_rate': 0.03,  # Reduced from 0.05
                'max_depth': 3,
                'min_samples_leaf': 25,  # Increased from 15
                'min_samples_split': 50,  # Added constraint
                'subsample': 0.7,
                'max_features': 'sqrt',
                'random_state': 42,
                'validation_fraction': 0.1,  # Early stopping support
                'n_iter_no_change': 20,  # Stop if no improvement
                'tol': 0.0001
            },
            'dt': {
                'max_depth': 4,
                'min_samples_leaf': 30,  # Increased from 25
                'min_samples_split': 60,  # Increased from 50
                'random_state': 42,
                'class_weight': 'balanced',
                'ccp_alpha': 0.001  # Minimal cost complexity pruning
            },
            'svm': {
                'C': 0.5,  # Increased regularization (lower C from 2.0)
                'kernel': 'rbf',
                'gamma': 'scale',
                'probability': True,
                'random_state': 42,
                'decision_function_shape': 'ovr',
                'class_weight': 'balanced'
            },
            'xgb': {
                'n_estimators': 150,  # Reduced from 300
                'learning_rate': 0.05,  # Reduced from 0.07
                'max_depth': 3,
                'subsample': 0.7,  # Reduced from 0.8
                'colsample_bytree': 0.7,  # Reduced from 0.8
                'reg_alpha': 0.5,  # Increased L1 regularization
                'reg_lambda': 2.0,  # Increased L2 regularization
                'min_child_weight': 3,  # Added constraint
                'random_state': 42,
                'objective': 'multi:softprob',
                'num_class': len(np.unique(y_train)),
                'tree_method': 'hist',
                'early_stopping_rounds': 20  # Early stopping
            }
        }

    # ----------------- SMOTE -----------------
    smote_kwargs = {'random_state': 42, 'k_neighbors': 3}

    logit = ImbPipeline([
        ("scaler", StandardScaler()),
        ("smote", SMOTE(**smote_kwargs)),
        ("logit", LogisticRegression(**model_params['logit']))
    ]).fit(X_train, y_train)

    rf = ImbPipeline([
        ("smote", SMOTE(**smote_kwargs)),
        ("rf", RandomForestClassifier(**model_params['rf']))
    ]).fit(X_train, y_train)

    gb = ImbPipeline([
        ("smote", SMOTE(**smote_kwargs)),
        ("gb", GradientBoostingClassifier(**model_params['gb']))
    ]).fit(X_train, y_train)

    dt = ImbPipeline([
        ("smote", SMOTE(**smote_kwargs)),
        ("dt", DecisionTreeClassifier(**model_params['dt']))
    ]).fit(X_train, y_train)

    svm = ImbPipeline([
        ("scaler", StandardScaler()),
        ("smote", SMOTE(**smote_kwargs)),
        ("svm", SVC(**model_params['svm']))
    ]).fit(X_train, y_train)

    models = {
        "Logistic Regression": logit,
        "Random Forest": rf,
        "Gradient Boosting": gb,
        "Decision Tree": dt,
        "SVM": svm,
    }

    if use_xgb:
        xgb = ImbPipeline([
            ("smote", SMOTE(**smote_kwargs)),
            ("xgb", XGBClassifier(**model_params['xgb']))
        ]).fit(X_train, y_train)
        models["XGBoost"] = xgb

    # ----------------- Ensemble -----------------
    ensemble_estimators = [
        ("rf", rf.named_steps["rf"]),
        ("gb", gb.named_steps["gb"]),
        ("svm", svm.named_steps["svm"]),
        ("logit", logit.named_steps["logit"])
    ]
    ensemble_weights = [2, 2, 1, 1]

    if use_xgb:
        ensemble_estimators.append(("xgb", xgb.named_steps["xgb"]))
        ensemble_weights.append(2)

    ensemble = VotingClassifier(
        estimators=ensemble_estimators,
        voting="soft",
        weights=ensemble_weights
    )
    ensemble.fit(X_train, y_train)
    models["Ensemble (Weighted)"] = ensemble

    # ----------------- Metrics -----------------
    results = {}
    confusion_mats = {}

    for name, m in models.items():
        # Predictions
        y_pred_train = m.predict(X_train)
        y_pred_val = m.predict(X_val)
        y_pred_test = m.predict(X_test)

        results[name] = {
            "Train Accuracy": accuracy_score(y_train, y_pred_train),
            "Val Accuracy": accuracy_score(y_val, y_pred_val),
            "Test Accuracy": accuracy_score(y_test, y_pred_test),

            "Val F1 (macro)": f1_score(y_val, y_pred_val, average="macro", zero_division=0),
            "Test F1 (macro)": f1_score(y_test, y_pred_test, average="macro", zero_division=0),

            "Val F1 (weighted)": f1_score(y_val, y_pred_val, average="weighted", zero_division=0),
            "Test F1 (weighted)": f1_score(y_test, y_pred_test, average="weighted", zero_division=0),
        }

        confusion_mats[name] = {
            "val": confusion_matrix(y_val, y_pred_val),
            "test": confusion_matrix(y_test, y_pred_test)
        }

    results_df = pd.DataFrame(results).T
    return models, results_df, confusion_mats

# ============================================================
#             FEATURE IMPORTANCE (MULTICLASS)
# ============================================================
def plot_feature_importance(models, feature_names, title_prefix="", top_n=15):
    """
    Display feature importance with improved visualization.
    top_n: Show only top N most important features
    """
    logit = models.get("Logistic Regression")
    rf = models.get("Random Forest")
    gb = models.get("Gradient Boosting")
    dt = models.get("Decision Tree")

    coef_matrix = logit.named_steps['logit'].coef_
    coef_mean_abs = np.mean(np.abs(coef_matrix), axis=0)

    logit_importance = pd.DataFrame({
        "Feature": feature_names,
        "Importance": coef_mean_abs
    }).sort_values("Importance", ascending=False).head(top_n)

    rf_importance = pd.DataFrame({
        "Feature": feature_names,
        "Importance": rf.named_steps['rf'].feature_importances_
    }).sort_values("Importance", ascending=False).head(top_n)

    gb_importance = pd.DataFrame({
        "Feature": feature_names,
        "Importance": gb.named_steps['gb'].feature_importances_
    }).sort_values("Importance", ascending=False).head(top_n)

    dt_importance = pd.DataFrame({
        "Feature": feature_names,
        "Importance": dt.named_steps['dt'].feature_importances_
    }).sort_values("Importance", ascending=False).head(top_n)

    fig, axes = plt.subplots(2, 2, figsize=(16, 10))

    # Sort in ascending order for better visualization
    logit_sorted = logit_importance.sort_values("Importance")
    rf_sorted = rf_importance.sort_values("Importance")
    gb_sorted = gb_importance.sort_values("Importance")
    dt_sorted = dt_importance.sort_values("Importance")

    axes[0, 0].barh(logit_sorted["Feature"], logit_sorted["Importance"], color='steelblue')
    axes[0, 0].set_title(f"{title_prefix}Logistic Regression (Top {top_n})", fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel("Mean |Coefficient|")

    axes[0, 1].barh(rf_sorted["Feature"], rf_sorted["Importance"], color='forestgreen')
    axes[0, 1].set_title(f"{title_prefix}Random Forest (Top {top_n})", fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel("Feature Importance")

    axes[1, 0].barh(gb_sorted["Feature"], gb_sorted["Importance"], color='darkorange')
    axes[1, 0].set_title(f"{title_prefix}Gradient Boosting (Top {top_n})", fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel("Feature Importance")

    axes[1, 1].barh(dt_sorted["Feature"], dt_sorted["Importance"], color='crimson')
    axes[1, 1].set_title(f"{title_prefix}Decision Tree (Top {top_n})", fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel("Feature Importance")

    plt.tight_layout()
    plt.show()


# ============================================================
#             CONFUSION MATRIX DISPLAY (MULTICLASS)
# ============================================================
def show_confusion_matrices(confusion_mats, results_df):
    """
    confusion_mats: dict {
        model_name: {
            "val":  cm_val,
            "test": cm_test
        }
    }
    """

    # Force exactly 3 rows × 2 columns = 6 windows
    n_rows = 3
    n_cols = 2

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 15))

    # Flatten axes for easy indexing
    axes = axes.reshape(n_rows, n_cols)

    # Only take the first 3 models (or pad if fewer)
    model_items = list(confusion_mats.items())[:3]

    for row_idx, (model_name, cms) in enumerate(model_items):

        # -------------------------
        # VALIDATION
        # -------------------------
        cm_val = cms["val"]
        disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val)
        disp_val.plot(cmap="Blues", ax=axes[row_idx, 0], colorbar=False)

        acc_val = results_df.loc[model_name, "Val Accuracy"]
        f1_val = results_df.loc[model_name, "Val F1 (macro)"]

        axes[row_idx, 0].set_title(
            f"{model_name} — Validation\nAcc={acc_val:.3f} | F1_macro={f1_val:.3f}",
            fontsize=11, fontweight="bold"
        )

        # -------------------------
        # TEST
        # -------------------------
        cm_test = cms["test"]
        disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test)
        disp_test.plot(cmap="Blues", ax=axes[row_idx, 1], colorbar=False)

        acc_test = results_df.loc[model_name, "Test Accuracy"]
        f1_test = results_df.loc[model_name, "Test F1 (macro)"]

        axes[row_idx, 1].set_title(
            f"{model_name} — Test\nAcc={acc_test:.3f} | F1_macro={f1_test:.3f}",
            fontsize=11, fontweight="bold"
        )

    plt.tight_layout()
    plt.show()

# ============================================================
#             ROC CURVES + MACRO AUC (MULTICLASS)
# ============================================================
def show_roc_curves(models, X, y, split_name="Test"):
    """
    Display ROC curves and macro AUC for a given split (val or test).
    """
    n_models = len(models)
    fig, axes = plt.subplots(1, n_models, figsize=(6*n_models, 5))

    if n_models == 1:
        axes = [axes]

    classes = sorted(np.unique(y))

    for ax, (name, model) in zip(axes, models.items()):

        if not hasattr(model, "predict_proba"):
            ax.set_title(f"{name}\n(No predict_proba)")
            ax.axis("off")
            continue

        y_proba = model.predict_proba(X)

        aucs = []
        for c in classes:
            y_true_bin = (y == c).astype(int)
            y_score = y_proba[:, c]

            fpr, tpr, _ = roc_curve(y_true_bin, y_score)
            auc_val = auc(fpr, tpr)
            aucs.append(auc_val)

            ax.plot(fpr, tpr, label=f"Class {c} (AUC={auc_val:.2f})", linewidth=2)

        macro_auc = np.mean(aucs)

        ax.plot([0, 1], [0, 1], "k--", alpha=0.5)
        ax.set_title(f"{name} — {split_name} ROC\nMacro AUC={macro_auc:.3f}", fontsize=11, fontweight='bold')
        ax.set_xlabel("False Positive Rate")
        ax.set_ylabel("True Positive Rate")
        ax.legend(loc='lower right', fontsize=8)
        ax.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
# -----------------------------
# Time-based split (Train / Val / Test)
# -----------------------------

SPLIT_YEAR = 2020     # Test starts here
VAL_YEAR   = 2018     # Validation starts here

# Masks
train_mask = df_filtered.index < VAL_YEAR
val_mask   = (df_filtered.index >= VAL_YEAR) & (df_filtered.index < SPLIT_YEAR)
test_mask  = df_filtered.index >= SPLIT_YEAR

# Apply masks
X_train = X[train_mask]
y_train = y[train_mask]

X_val = X[val_mask]
y_val = y[val_mask]

X_test = X[test_mask]
y_test = y[test_mask]

# Training the model 

In [ ]:
models, summary_df, confusion_mats = train_all_models(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test
)

print(summary_df)

plot_feature_importance(models, X_train.columns.tolist())

# Show TEST confusion matrices
show_confusion_matrices(confusion_mats, summary_df)

# ROC curves on TEST
show_roc_curves(models, X_test, y_test)


### Reduced Global Set

In [ ]:
#reduced variables set
selected_features = ['TM_RPCH', 'NGDP_RPCH', 'TX_RPCH', 'BCA_NGDPD', 'PCPIPCH', 'NGDPRPC']

In [ ]:
# -----------------------------------------
# YEAR-BASED SPLIT (same logic everywhere)
# -----------------------------------------
TRAIN_END = 2018     # Train: < 2018 (up to 2017)
VAL_END   = 2021     # Validation: 2018-2020
                     # Test: >= 2021 (2021-2025)

train_mask = X.index < TRAIN_END
val_mask   = (X.index >= TRAIN_END) & (X.index < VAL_END)
test_mask  = X.index >= VAL_END

X_train = X[train_mask]
y_train = y[train_mask]

X_val = X[val_mask]
y_val = y[val_mask]

X_test = X[test_mask]
y_test = y[test_mask]
print(f"Train: {len(X_train)} samples (years < {TRAIN_END} [up to 2017])")
print(f"Validation: {len(X_val)} samples ({TRAIN_END}-{VAL_END-1} [2018-2020])")
print(f"Test: {len(X_test)} samples (years >= {VAL_END} [2021-2025])")
print(f"Test: {len(X_test)} samples (years >= {VAL_END})")

# -----------------------------------------
# REDUCE FEATURES FOR ALL SPLITS
# -----------------------------------------
X_train_reduced = X_train[selected_features]
X_val_reduced   = X_val[selected_features]
X_test_reduced  = X_test[selected_features]

# -----------------------------------------
# TRAIN MODELS WITH REDUCED FEATURES
# -----------------------------------------
models_reduced, summary_df_reduced, confusion_mats_reduced = train_all_models(
    X_train_reduced, y_train,
    X_val_reduced,   y_val,
    X_test_reduced,  y_test
)

# -----------------------------------------
# METRICS TABLE
# -----------------------------------------
print(summary_df_reduced)

# -----------------------------------------
# FEATURE IMPORTANCE
# -----------------------------------------
plot_feature_importance(
    models_reduced,
    feature_names=selected_features,
    title_prefix="Reduced Features - "
)

# -----------------------------------------
# COMBINED VALIDATION + TEST CONFUSION MATRICES
# -----------------------------------------
show_confusion_matrices(confusion_mats_reduced, summary_df_reduced)

# -----------------------------------------
# ROC CURVES (TEST SET)
# -----------------------------------------
show_roc_curves(models_reduced, X_test_reduced, y_test, split_name="Test")


### Split Dataset

In [ ]:
# Map countries to continents (same logic as before)
try:
    import pycountry
    import pycountry_convert as pc
    
    def country_to_continent(name):
        try:
            lookup_name = name.replace('_', ' ')
            country = pycountry.countries.lookup(lookup_name)
            alpha2 = country.alpha_2
            cc = pc.country_alpha2_to_continent_code(alpha2)
            continent_map = {
                'AF': 'Africa',
                'AS': 'Asia',
                'EU': 'Europe',
                'NA': 'North_America',
                'OC': 'Oceania',
                'SA': 'South_America'
            }
            return continent_map.get(cc, 'Unknown')
        except Exception:
            return 'Unknown'
except ImportError:
    # Fallback mapping for common countries (extend as needed)
    fallback = {
    # --- North America ---
    'United_States': 'North_America',
    'Canada': 'North_America',
    'Mexico': 'North_America',
    'Costa_Rica': 'North_America',
    'Dominican_Republic': 'North_America',
    'Panama': 'North_America',
    'Trinidad_and_Tobago': 'North_America',
    'Barbados': 'North_America',
    'St._Kitts_and_Nevis': 'North_America',
    'St._Lucia': 'North_America',
    'St._Vincent_and_the_Grenadines': 'North_America',
    'Grenada': 'North_America',
    'Antigua_and_Barbuda': 'North_America',
    'The_Bahamas': 'North_America',
    'Belize': 'North_America',
    'Haiti': 'North_America',
    'Jamaica': 'North_America',
    'Puerto_Rico': 'North_America',
    'Aruba': 'North_America',
    'Dominica': 'North_America',
    'Nicaragua': 'North_America',
    'Guatemala': 'North_America',
    'Honduras': 'North_America',
    'El_Salvador': 'North_America',

    # --- South America ---
    'Brazil': 'South_America',
    'Argentina': 'South_America',
    'Chile': 'South_America',
    'Colombia': 'South_America',
    'Peru': 'South_America',
    'Venezuela': 'South_America',
    'Bolivia': 'South_America',
    'Paraguay': 'South_America',
    'Uruguay': 'South_America',
    'Ecuador': 'South_America',
    'Guyana': 'South_America',
    'Suriname': 'South_America',

    # --- Europe ---
    'Germany': 'Europe',
    'France': 'Europe',
    'United_Kingdom': 'Europe',
    'Italy': 'Europe',
    'Spain': 'Europe',
    'Russia': 'Europe',
    'Turkey': 'Europe',
    'Türkiye': 'Europe',  # Modern spelling of Turkey
    'Poland': 'Europe',
    'Albania': 'Europe',
    'Austria': 'Europe',
    'Belgium': 'Europe',
    'Bosnia_and_Herzegovina': 'Europe',
    'Bulgaria': 'Europe',
    'Croatia': 'Europe',
    'Cyprus': 'Europe',
    'Czech_Republic': 'Europe',
    'Denmark': 'Europe',
    'Estonia': 'Europe',
    'Finland': 'Europe',
    'Hungary': 'Europe',
    'Iceland': 'Europe',
    'Ireland': 'Europe',
    'Latvia': 'Europe',
    'Lithuania': 'Europe',
    'Luxembourg': 'Europe',
    'Malta': 'Europe',
    'Netherlands': 'Europe',
    'North_Macedonia': 'Europe',
    'Norway': 'Europe',
    'Portugal': 'Europe',
    'Romania': 'Europe',
    'Serbia': 'Europe',
    'Slovak_Republic': 'Europe',
    'Slovenia': 'Europe',
    'Sweden': 'Europe',
    'Switzerland': 'Europe',
    'Greece': 'Europe',
    'Kosovo': 'Europe',
    'Andorra': 'Europe',
    'San_Marino': 'Europe',
    'Monaco': 'Europe' if 'Monaco' in locals() else None,
    'Belarus': 'Europe',
    'Moldova': 'Europe',
    'Montenegro': 'Europe',
    'Ukraine': 'Europe',

    # --- Asia ---
    'China': 'Asia',
    'India': 'Asia',
    'Japan': 'Asia',
    'Afghanistan': 'Asia',
    'Korea': 'Asia',
    'Indonesia': 'Asia',
    'Thailand': 'Asia',
    'Vietnam': 'Asia',
    'Islamic_Republic_of_Iran': 'Asia',
    'Israel': 'Asia',
    'Jordan': 'Asia',
    'Kazakhstan': 'Asia',
    'Lebanon': 'Asia',
    'Pakistan': 'Asia',
    'Saudi_Arabia': 'Asia',
    'Syria': 'Asia',
    'Taiwan_Province_of_China': 'Asia',
    'Oman': 'Asia',
    'Yemen': 'Asia',
    'United_Arab_Emirates': 'Asia',
    'Armenia': 'Asia',
    'Azerbaijan': 'Asia',
    'Georgia': 'Asia',
    'Qatar': 'Asia',
    'Bahrain': 'Asia',
    'Kuwait': 'Asia',
    'Nepal': 'Asia',
    'Sri_Lanka': 'Asia',
    'Bangladesh': 'Asia',
    'Cambodia': 'Asia',
    'Myanmar': 'Asia',
    'Lao_P.D.R.': 'Asia',
    'Bhutan': 'Asia',
    'Mongolia': 'Asia',
    'Malaysia': 'Asia',
    'Philippines': 'Asia',
    'Singapore': 'Asia',
    'Timor_Leste': 'Asia',
    'Brunei_Darussalam': 'Asia',
    'Hong_Kong_SAR': 'Asia',
    'Macao_SAR': 'Asia',
    'West_Bank_and_Gaza': 'Asia',  # Palestinian territories
    'Iraq': 'Asia',
    'Kyrgyz_Republic': 'Asia',
    'Tajikistan': 'Asia',
    'Turkmenistan': 'Asia',
    'Uzbekistan': 'Asia',
    'Maldives': 'Asia',

    # --- Africa ---
    'South_Africa': 'Africa',
    'Nigeria': 'Africa',
    'Egypt': 'Africa',
    'Zimbabwe': 'Africa',
    'Kenya': 'Africa',
    'Ethiopia': 'Africa',
    'Morocco': 'Africa',
    'Algeria': 'Africa',
    'Cabo_Verde': 'Africa',
    'Seychelles': 'Africa',
    'Botswana': 'Africa',
    'Cameroon': 'Africa',
    'Djibouti': 'Africa',
    'Equatorial_Guinea': 'Africa',
    'Eswatini': 'Africa',
    'Lesotho': 'Africa',
    'Mali': 'Africa',
    'Mauritania': 'Africa',
    'Namibia': 'Africa',
    'Niger': 'Africa',
    'Zambia': 'Africa',
    'Ghana': 'Africa',
    'Uganda': 'Africa',
    'Tanzania': 'Africa',
    'Rwanda': 'Africa',
    'Burundi': 'Africa',
    'Somalia': 'Africa',
    'Sudan': 'Africa',
    'South_Sudan': 'Africa',
    'Chad': 'Africa',
    'Benin': 'Africa',
    'Liberia': 'Africa',
    'Guinea': 'Africa',
    'Guinea_Bissau': 'Africa',
    'Sierra_Leone': 'Africa',
    'Togo': 'Africa',
    'Angola': 'Africa',
    'Mozambique': 'Africa',
    'Madagascar': 'Africa',
    'Comoros': 'Africa',
    'São_Tomé_and_Príncipe': 'Africa',
    'Republic_of_Congo': 'Africa',
    'Democratic_Republic_of_the_Congo': 'Africa',
    'Eritrea': 'Africa',
    'Libya': 'Africa',
    'Tunisia': 'Africa',
    'Gabon': 'Africa',
    'Central_African_Republic': 'Africa',
    'The_Gambia': 'Africa',
    'Senegal': 'Africa',
    'Mauritius': 'Africa',
    'Sao_Tome_and_Principe': 'Africa',
    'Côte_dIvoire': 'Africa',  # Ivory Coast
    'Burkina_Faso': 'Africa',
    'Malawi': 'Africa',
    'Congo_Republic': 'Africa',

    # --- Oceania ---
    'Australia': 'Oceania',
    'New_Zealand': 'Oceania',
    'Fiji': 'Oceania',
    'Tonga': 'Oceania',
    'Samoa': 'Oceania',
    'Vanuatu': 'Oceania',
    'Papua_New_Guinea': 'Oceania',
    'Solomon_Islands': 'Oceania',
    'Kiribati': 'Oceania',
    'Marshall_Islands': 'Oceania',
    'Micronesia': 'Oceania',
    'Nauru': 'Oceania',
    'Tuvalu': 'Oceania',
    'Palau': 'Oceania'
}

    def country_to_continent(name):
        return fallback.get(name.replace(' ', '_'), 'Unknown')

# --- Add Continent column ---
df_filtered_copy = df_pivot.copy()
df_filtered_copy['Continent'] = df_filtered_copy['Country'].astype(str).apply(country_to_continent)

# --- Map continents to economy groups ---
continent_to_economy = {
    'Europe': 'Upper_Economies',
    'North_America': 'Upper_Economies',
    'Oceania': 'Upper_Economies',
    'Africa': 'Lower_Economies',
    'Asia': 'Lower_Economies',
    'South_America': 'Lower_Economies'
}

df_filtered_copy['EconomyGroup'] = df_filtered_copy['Continent'].map(continent_to_economy)

# --- Create Lower and Upper economy DataFrames ---
df_Lower_Economies = df_filtered_copy[df_filtered_copy['EconomyGroup'] == 'Lower_Economies'].drop(columns=['Continent','EconomyGroup'])
df_Upper_Economies = df_filtered_copy[df_filtered_copy['EconomyGroup'] == 'Upper_Economies'].drop(columns=['Continent','EconomyGroup'])

# --- Print summary ---
print("=" * 80)
print(" " * 25 + "ECONOMY GROUP SUMMARY")
print("=" * 80)

print(f"\nTotal DataFrames created:")
print(f"  • Lower Economies: {len(df_Lower_Economies)} rows")
print(f"  • Upper Economies: {len(df_Upper_Economies)} rows")
print(f"  • Total: {len(df_filtered_copy)} rows")

# --- Show RecessionRisk distribution per economy group ---
print("\n" + "=" * 80)
print("RECESSION RISK DISTRIBUTION BY ECONOMY GROUP")
print("=" * 80)

print("\n📊 LOWER ECONOMIES (Africa, Asia, South America):")
print("-" * 80)
lower_risk_dist = df_Lower_Economies['RecessionRisk'].value_counts().sort_index()
print(lower_risk_dist)
print(f"\nTotal Lower Economy samples: {len(df_Lower_Economies)}")

print("\nDetailed breakdown:")
for risk_class in range(4):
    count = lower_risk_dist.get(risk_class, 0)
    pct = (count / len(df_Lower_Economies)) * 100 if len(df_Lower_Economies) > 0 else 0
    risk_label = ['No Risk', 'Mild', 'Moderate', 'High'][risk_class]
    print(f"  Class {risk_class} ({risk_label:>8}): {count:>5} samples ({pct:>6.2f}%)")

print("\n" + "-" * 80)
print("\n📊 UPPER ECONOMIES (Europe, North America, Oceania):")
print("-" * 80)
upper_risk_dist = df_Upper_Economies['RecessionRisk'].value_counts().sort_index()
print(upper_risk_dist)
print(f"\nTotal Upper Economy samples: {len(df_Upper_Economies)}")

print("\nDetailed breakdown:")
for risk_class in range(4):
    count = upper_risk_dist.get(risk_class, 0)
    pct = (count / len(df_Upper_Economies)) * 100 if len(df_Upper_Economies) > 0 else 0
    risk_label = ['No Risk', 'Mild', 'Moderate', 'High'][risk_class]
    print(f"  Class {risk_class} ({risk_label:>8}): {count:>5} samples ({pct:>6.2f}%)")

# --- Comparative Analysis ---
print("\n" + "=" * 80)
print("COMPARATIVE ANALYSIS: LOWER vs UPPER ECONOMIES")
print("=" * 80)

print(f"\n{'Risk Class':<15} {'Lower Econ':<20} {'Upper Econ':<20} {'Difference':<15}")
print("-" * 80)

for risk_class in range(4):
    lower_count = lower_risk_dist.get(risk_class, 0)
    upper_count = upper_risk_dist.get(risk_class, 0)
    
    lower_pct = (lower_count / len(df_Lower_Economies)) * 100 if len(df_Lower_Economies) > 0 else 0
    upper_pct = (upper_count / len(df_Upper_Economies)) * 100 if len(df_Upper_Economies) > 0 else 0
    
    diff = lower_pct - upper_pct
    risk_label = ['No Risk', 'Mild', 'Moderate', 'High'][risk_class]
    
    print(f"{risk_label:<15} {lower_count:>5} ({lower_pct:>5.1f}%)     {upper_count:>5} ({upper_pct:>5.1f}%)     {diff:>+6.1f}%")

# --- Visualization ---
print("\n" + "=" * 80)
print("VISUALIZATION: Recession Risk by Economy Group")
print("=" * 80)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Lower Economies
lower_counts = [lower_risk_dist.get(i, 0) for i in range(4)]
axes[0].bar(range(4), lower_counts, color=['green', 'yellow', 'orange', 'red'], 
            edgecolor='black', linewidth=1.5)
axes[0].set_xlabel('RecessionRisk Class', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Lower Economies\n(Africa, Asia, South America)', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(4))
axes[0].set_xticklabels(['No Risk', 'Mild', 'Moderate', 'High'])
axes[0].grid(axis='y', alpha=0.3)

# Add count labels
for i, count in enumerate(lower_counts):
    if count > 0:
        axes[0].text(i, count + max(lower_counts)*0.02, str(count), 
                    ha='center', va='bottom', fontsize=11, fontweight='bold')

# Upper Economies
upper_counts = [upper_risk_dist.get(i, 0) for i in range(4)]
axes[1].bar(range(4), upper_counts, color=['green', 'yellow', 'orange', 'red'], 
            edgecolor='black', linewidth=1.5)
axes[1].set_xlabel('RecessionRisk Class', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[1].set_title('Upper Economies\n(Europe, North America, Oceania)', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(4))
axes[1].set_xticklabels(['No Risk', 'Mild', 'Moderate', 'High'])
axes[1].grid(axis='y', alpha=0.3)

# Add count labels
for i, count in enumerate(upper_counts):
    if count > 0:
        axes[1].text(i, count + max(upper_counts)*0.02, str(count), 
                    ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# --- Countries per economy group ---
print("\n" + "=" * 80)
print("COUNTRIES BY ECONOMY GROUP")
print("=" * 80)

print("\nLower Economies Countries:")
lower_countries = sorted(df_Lower_Economies['Country'].unique())
print(f"  Total: {len(lower_countries)} countries")
print(f"  {', '.join(lower_countries[:10])}...")  # Show first 10

print("\nUpper Economies Countries:")
upper_countries = sorted(df_Upper_Economies['Country'].unique())
print(f"  Total: {len(upper_countries)} countries")
print(f"  {', '.join(upper_countries[:10])}...")  # Show first 10

print("\n" + "=" * 80)

# 6. Economy-Specific Analysis

## Upper Economies - Full Features

In [ ]:
# Prepare data
X = df_Upper_Economies.drop(columns=["RecessionRisk", "Country"])
y = df_Upper_Economies["RecessionRisk"]

# -----------------------------
# TIME-BASED SPLIT (Train / Val / Test)
# -----------------------------
TRAIN_END = 2018     # Train: < 2018 (up to 2017)
VAL_END   = 2021     # Validation: 2018-2020
                     # Test: >= 2021 (2021-2025)

train_mask = X.index < TRAIN_END
val_mask   = (X.index >= TRAIN_END) & (X.index < VAL_END)
test_mask  = X.index >= VAL_END

X_train = X[train_mask]
y_train = y[train_mask]

X_val = X[val_mask]
y_val = y[val_mask]

X_test = X[test_mask]
y_test = y[test_mask]

print(f"Train: {len(X_train)} samples (years < {TRAIN_END} [up to 2017])")
print(f"Validation: {len(X_val)} samples ({TRAIN_END}-{VAL_END-1} [2018-2020])")
print(f"Test: {len(X_test)} samples (years >= {VAL_END} [2021-2025])")
models_upper, summary_df_upper, confusion_mats_upper = train_all_models(
    X_train, y_train,
    X_val,   y_val,
    X_test,  y_test
)
print(summary_df_upper)
plot_feature_importance(
    models_upper,
    X_train.columns.tolist(),
    title_prefix="Upper Economies - "
)
show_confusion_matrices(confusion_mats_upper, summary_df_upper)
show_roc_curves(models_upper, X_test, y_test)

## Lower Economies - Full Features

In [ ]:
# -----------------------------------------
# PREPARE DATA FOR LOWER ECONOMIES
# -----------------------------------------
X = df_Lower_Economies.drop(columns=["RecessionRisk", "Country"])
y = df_Lower_Economies["RecessionRisk"]

# -----------------------------------------
# YEAR-BASED SPLIT (same logic as Upper Economies)
# -----------------------------------------
TRAIN_END = 2018     # Train: < 2018 (up to 2017)
VAL_END   = 2021     # Validation: 2018-2020
                     # Test: >= 2021 (2021-2025)

train_mask = X.index < TRAIN_END
val_mask   = (X.index >= TRAIN_END) & (X.index < VAL_END)
test_mask  = X.index >= VAL_END

X_train = X[train_mask]
y_train = y[train_mask]

X_val = X[val_mask]
y_val = y[val_mask]

X_test = X[test_mask]
y_test = y[test_mask]
print(f"Train: {len(X_train)} samples (years < {TRAIN_END} [up to 2017])")
print(f"Validation: {len(X_val)} samples ({TRAIN_END}-{VAL_END-1} [2018-2020])")
print(f"Test: {len(X_test)} samples (years >= {VAL_END} [2021-2025])")
print(f"Test: {len(X_test)} samples (years >= {VAL_END})")

# -----------------------------------------
# TRAIN MODELS (NOW WITH VALIDATION)
# -----------------------------------------
models_lower, summary_df_lower, confusion_mats_lower = train_all_models(
    X_train, y_train,
    X_val,   y_val,
    X_test,  y_test
)

# -----------------------------------------
# METRICS TABLE
# -----------------------------------------
print(summary_df_lower)

# -----------------------------------------
# FEATURE IMPORTANCE
# -----------------------------------------
plot_feature_importance(
    models_lower,
    X_train.columns.tolist(),
    title_prefix="Lower Economies - "
)

# -----------------------------------------
# COMBINED VALIDATION + TEST CONFUSION MATRICES
# -----------------------------------------
show_confusion_matrices(confusion_mats_lower, summary_df_lower)

# -----------------------------------------
# ROC CURVES (TEST SET)
# -----------------------------------------
show_roc_curves(models_lower, X_test, y_test, split_name="Test")


## Upper Economies - Reduced Features

In [ ]:
# -----------------------------------------
# PREPARE DATA (REDUCED FEATURES)
# -----------------------------------------
X_upper = df_Upper_Economies[selected_features]
y_upper = df_Upper_Economies["RecessionRisk"]

# -----------------------------------------
# YEAR-BASED SPLIT (same logic everywhere)
# -----------------------------------------
TRAIN_END = 2018     # Train: < 2018 (up to 2017)
VAL_END   = 2021     # Validation: 2018-2020
                     # Test: >= 2021 (2021-2025)

train_mask = X_upper.index < TRAIN_END
val_mask   = (X_upper.index >= TRAIN_END) & (X_upper.index < VAL_END)
test_mask  = X_upper.index >= VAL_END

X_train_upper = X_upper[train_mask]
y_train_upper = y_upper[train_mask]

X_val_upper = X_upper[val_mask]
y_val_upper = y_upper[val_mask]

X_test_upper = X_upper[test_mask]
y_test_upper = y_upper[test_mask]
print(f"Train: {len(X_train_upper)} samples (years < {TRAIN_END} [up to 2017])")
print(f"Validation: {len(X_val_upper)} samples ({TRAIN_END}-{VAL_END-1} [2018-2020])")
print(f"Test: {len(X_test_upper)} samples (years >= {VAL_END} [2021-2025])")
print(f"Test: {len(X_test_upper)} samples (years >= {VAL_END})")

# -----------------------------------------
# TRAIN MODELS (REDUCED FEATURES)
# -----------------------------------------
models_upper_reduced, summary_df_upper_reduced, confusion_mats_upper_reduced = train_all_models(
    X_train_upper, y_train_upper,
    X_val_upper,   y_val_upper,
    X_test_upper,  y_test_upper
)

# -----------------------------------------
# METRICS TABLE
# -----------------------------------------
print("Upper Economies Accuracy (Reduced Features):")
print(summary_df_upper_reduced)

# -----------------------------------------
# FEATURE IMPORTANCE
# -----------------------------------------
plot_feature_importance(
    models_upper_reduced,
    feature_names=selected_features,
    title_prefix="Upper Economies - Reduced Features - "
)

# -----------------------------------------
# COMBINED VALIDATION + TEST CONFUSION MATRICES
# -----------------------------------------
show_confusion_matrices(confusion_mats_upper_reduced, summary_df_upper_reduced)

# -----------------------------------------
# ROC CURVES (TEST SET)
# -----------------------------------------
show_roc_curves(models_upper_reduced, X_test_upper, y_test_upper, split_name="Test")


## Lower Economies - Reduced Features

In [ ]:
# -----------------------------------------
# PREPARE DATA (REDUCED FEATURES)
# -----------------------------------------
X_lower = df_Lower_Economies[selected_features]
y_lower = df_Lower_Economies["RecessionRisk"]

# -----------------------------------------
# YEAR-BASED SPLIT (same logic everywhere)
# -----------------------------------------
TRAIN_END = 2018     # Train: < 2018 (up to 2017)
VAL_END   = 2021     # Validation: 2018-2020
                     # Test: >= 2021 (2021-2025)

train_mask = X_lower.index < TRAIN_END
val_mask   = (X_lower.index >= TRAIN_END) & (X_lower.index < VAL_END)
test_mask  = X_lower.index >= VAL_END

X_train_lower = X_lower[train_mask]
y_train_lower = y_lower[train_mask]

X_val_lower = X_lower[val_mask]
y_val_lower = y_lower[val_mask]

X_test_lower = X_lower[test_mask]
y_test_lower = y_lower[test_mask]
print(f"Train: {len(X_train_lower)} samples (years < {TRAIN_END} [up to 2017])")
print(f"Validation: {len(X_val_lower)} samples ({TRAIN_END}-{VAL_END-1} [2018-2020])")
print(f"Test: {len(X_test_lower)} samples (years >= {VAL_END} [2021-2025])")
print(f"Test: {len(X_test_lower)} samples (years >= {VAL_END})")

# -----------------------------------------
# TRAIN MODELS (REDUCED FEATURES)
# -----------------------------------------
models_lower_reduced, summary_df_lower_reduced, confusion_mats_lower_reduced = train_all_models(
    X_train_lower, y_train_lower,
    X_val_lower,   y_val_lower,
    X_test_lower,  y_test_lower
)

# -----------------------------------------
# METRICS TABLE
# -----------------------------------------
print("Lower Economies Accuracy (Reduced Features):")
print(summary_df_lower_reduced)

# -----------------------------------------
# FEATURE IMPORTANCE
# -----------------------------------------
plot_feature_importance(
    models_lower_reduced,
    feature_names=selected_features,
    title_prefix="Lower Economies - Reduced Features - "
)

# -----------------------------------------
# COMBINED VALIDATION + TEST CONFUSION MATRICES
# -----------------------------------------
show_confusion_matrices(confusion_mats_lower_reduced, summary_df_lower_reduced)

# -----------------------------------------
# ROC CURVES (TEST SET)
# -----------------------------------------
show_roc_curves(models_lower_reduced, X_test_lower, y_test_lower, split_name="Test")


# LTSM Neural Networks

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Core layers
from tensorflow.keras.layers import (
    Input,
    Dense,
    LSTM,
    GRU,
    Dropout,
    BatchNormalization,
    LayerNormalization,
    Bidirectional
)

# Advanced sequence layers
from tensorflow.keras.layers import (
    Conv1D,
    GlobalAveragePooling1D,
    GlobalMaxPooling1D,
    MultiHeadAttention
)

# Model utilities
from tensorflow.keras.models import Sequential, Model

# Callbacks
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    ModelCheckpoint
)

# Optimizers
from tensorflow.keras.optimizers import Adam, RMSprop

# Losses & metrics
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy


In [ ]:
df_until_2025 = df_pivot[df_pivot.index <= 2025]
df_after_2025 = df_pivot[df_pivot.index > 2025]


In [ ]:
df_until_2025

In [ ]:
df_after_2025

In [ ]:
def reshape_for_lstm(df, window=3, min_sequences=1, return_feature_names=False):
    """
    Convert df_pivot (Year index, Country column) into LSTM-ready sequences.
    
    Args:
        df: DataFrame with Year index and Country column
        window: Number of historical years to use as input
        min_sequences: Minimum sequences required per country (default 1)
        return_feature_names: If True, return feature column names
        
    Returns:
        X_seq: (num_sequences, window, num_features)
        y_seq: (num_sequences,)
        countries_seq: country for each sequence
        years_seq: ending year for each sequence
        feature_names: (optional) list of feature column names
    """
    df = df.copy()
    
    # Handle index - reset if needed for sorting
    if isinstance(df.index, pd.RangeIndex):
        raise ValueError("DataFrame must have Year as index")
    
    # Reset index to make sorting easier, then sort
    df_sorted = df.reset_index().sort_values(["Country", df.index.name])
    year_col = df.index.name or "Year"
    
    # Define features (exclude target and identifiers)
    feature_cols = [c for c in df_sorted.columns 
                   if c not in ["RecessionRisk", "Country", year_col]]
    
    X_seq = []
    y_seq = []
    countries_seq = []
    years_seq = []
    
    # Group by country
    for country, group in df_sorted.groupby("Country"):
        group = group.sort_values(year_col).reset_index(drop=True)
        
        # Check if country has enough data
        if len(group) < window + 1:
            print(f"Warning: {country} has only {len(group)} years, need {window + 1}. Skipping.")
            continue
        
        X_values = group[feature_cols].values
        y_values = group["RecessionRisk"].values
        years = group[year_col].values
        
        # Sliding window
        num_sequences = len(group) - window
        for i in range(num_sequences):
            X_seq.append(X_values[i:i+window])
            y_seq.append(y_values[i+window])
            countries_seq.append(country)
            years_seq.append(years[i+window])
    
    if len(X_seq) < min_sequences:
        raise ValueError(f"Only generated {len(X_seq)} sequences, need at least {min_sequences}")
    
    results = (
        np.array(X_seq),
        np.array(y_seq),
        np.array(countries_seq),
        np.array(years_seq)
    )
    
    if return_feature_names:
        results = results + (feature_cols,)
    
    return results

In [ ]:
WINDOW = 3

X_seq, y_seq, countries_seq, years_seq = reshape_for_lstm(
    df_until_2025,
    window=WINDOW
)

print("X_seq shape:", X_seq.shape)
print("y_seq shape:", y_seq.shape)
print("Example sequence shape:", X_seq[0].shape)


In [ ]:
# -----------------------------
# 1. First split: Train vs Test
# -----------------------------
SPLIT_YEAR = 2020

train_mask = years_seq <= SPLIT_YEAR
test_mask  = years_seq > SPLIT_YEAR

X_train_full = X_seq[train_mask]
y_train_full = y_seq[train_mask]
years_train_full = years_seq[train_mask]

X_test = X_seq[test_mask]
y_test = y_seq[test_mask]
years_test = years_seq[test_mask]

# -----------------------------
# 2. Second split: Train vs Validation
# -----------------------------
VAL_YEAR = 2017  # You can adjust this

train_mask_2 = years_train_full <= VAL_YEAR
val_mask_2   = (years_train_full > VAL_YEAR) & (years_train_full <= SPLIT_YEAR)

X_train = X_train_full[train_mask_2]
y_train = y_train_full[train_mask_2]

X_val   = X_train_full[val_mask_2]
y_val   = y_train_full[val_mask_2]

# -----------------------------
# 3. Print summary
# -----------------------------
print(f"Train sequences: {len(X_train)} (years {years_train_full[train_mask_2].min()}–{years_train_full[train_mask_2].max()})")
print(f"Validation sequences: {len(X_val)} (years {years_train_full[val_mask_2].min()}–{years_train_full[val_mask_2].max()})")
print(f"Test sequences: {len(X_test)} (years {years_test.min()}–{years_test.max()})")

total = len(X_train) + len(X_val) + len(X_test)
print(f"Split: {len(X_train)/total*100:.1f}% train, {len(X_val)/total*100:.1f}% val, {len(X_test)/total*100:.1f}% test")


In [ ]:
# -----------------------------
# 1. SCALE FEATURES (minimal)
# -----------------------------
scaler = StandardScaler()

# Fit only on TRAIN
X_train_scaled = scaler.fit_transform(
    X_train.reshape(-1, X_train.shape[-1])
).reshape(X_train.shape)

# Transform VAL + TEST
X_val_scaled = scaler.transform(
    X_val.reshape(-1, X_val.shape[-1])
).reshape(X_val.shape)

X_test_scaled = scaler.transform(
    X_test.reshape(-1, X_test.shape[-1])
).reshape(X_test.shape)

num_classes = len(np.unique(y_seq))

# -----------------------------
# 2. LSTM MODEL
# -----------------------------
model = keras.Sequential([
    keras.layers.Input(shape=(WINDOW, X_train.shape[2])),
    keras.layers.LSTM(32),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# 3. TRAIN (now uses VALIDATION SET)
# -----------------------------
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=20,
    batch_size=32,
    verbose=1
)

# -----------------------------
# 4. EVALUATE ON TEST SET
# -----------------------------
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

# -----------------------------
# 5. PREDICT ON TEST SET
# -----------------------------
y_pred = np.argmax(model.predict(X_test_scaled), axis=1)


In [ ]:
# Generate predictions for all splits
y_train_pred = np.argmax(model.predict(X_train_scaled), axis=1)
y_val_pred   = np.argmax(model.predict(X_val_scaled), axis=1)
y_test_pred  = y_pred

# Quick summary
print(f"\nAccuracy - Train: {accuracy_score(y_train, y_train_pred):.3f} | Val: {accuracy_score(y_val, y_val_pred):.3f} | Test: {accuracy_score(y_test, y_test_pred):.3f}")
print(f"F1 (macro) - Train: {f1_score(y_train, y_train_pred, average='macro', zero_division=0):.3f} | Val: {f1_score(y_val, y_val_pred, average='macro', zero_division=0):.3f} | Test: {f1_score(y_test, y_test_pred, average='macro', zero_division=0):.3f}")


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------
# CONFUSION MATRICES
# -----------------------------
cm_train = confusion_matrix(y_train, y_train_pred)
cm_val   = confusion_matrix(y_val, y_val_pred)
cm_test  = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(18, 5))

# Train
plt.subplot(1, 3, 1)
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Train Confusion Matrix")

# Validation
plt.subplot(1, 3, 2)
sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Validation Confusion Matrix")

# Test
plt.subplot(1, 3, 3)
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Test Confusion Matrix")

plt.tight_layout()
plt.show()

# -----------------------------
# TRAINING CURVES
# -----------------------------
plt.figure(figsize=(12, 4))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve")
plt.legend()

plt.tight_layout()
plt.show()
